# 3. Interactive Debugging tool

Objective: This section of the project deals with a code implementing interactive widgets to perform the following two tasks  

1. Allows the users to select a game using a slider object and print the match summary ie., team names, number of golas by each team and the winner of the match

2. Allows the user to select any game with it's ID, aceess all the goal/Shot events of that selected game to siplay the position of the punk on the rink.

Refer to ift6758/data/question_2_4.py for functions used to import the data.

This takes a while to run, so run it only once, save output as csv and import the csv for future uses

In [11]:
import json

In [21]:
df = tidy_df_loop()

finished 2018 season
finished 2019 season


In [22]:
df.to_csv('tidy_df.csv')

In [23]:
import pandas as pd
df = pd.read_csv('tidy_df.csv')

In [24]:
import ipywidgets as widgets
from ipywidgets import interactive

In [25]:
import importlib
import matplotlib.pyplot as plt

Code for match summary

In [215]:
df['game_id_numeric'] = pd.to_numeric(df['game_id'])
list_game_id = df['game_id_numeric'].unique().tolist()

def Game_ID(val):
    match_summary =[]
    match_summary_col = ['Home Team','Home Team Score','Home Team Attempts','Away Team','Away Team Score','Away Team Attempts','Winner']
    count = 0
    with open(f'{data_folder_path}{val}.json','r') as f:
        data_Q3 = json.load(f)
        df_Q3 = pd.DataFrame(data_Q3)
        
    if df_Q3['liveData']['linescore']['hasShootout'] is not True:
        Home_Team = df_Q3['gameData']['teams']['home']['name']
        Away_Team = df_Q3['gameData']['teams']['away']['name']
        Home_Team_Score = df_Q3['liveData']['linescore']['teams']['home']['goals']
        Home_Team_Attempts = df_Q3['liveData']['linescore']['teams']['home']['shotsOnGoal']
        Away_Team_Score = df_Q3['liveData']['linescore']['teams']['away']['goals']
        Away_Team_Attempts = df_Q3['liveData']['linescore']['teams']['away']['shotsOnGoal']
        if Home_Team_Score > Away_Team_Score:
            winner = Home_Team
        else:
            winner = Away_Team
        match_summary.append([Home_Team,Home_Team_Score,Home_Team_Attempts,Away_Team,Away_Team_Score,Away_Team_Attempts,winner])
        match_summary = pd.DataFrame(match_summary, columns = match_summary_col)
        print("Game summary of ",val," between",Home_Team," and ",Away_Team)

        return(match_summary)
    

Game_ID = widgets.interact(Game_ID, val=widgets.SelectionSlider(description='Game_ID',options=list_game_id))


interactive(children=(SelectionSlider(description='Game_ID', options=(2018020001, 2018020002, 2018020003, 2018…

This portion of the code allows the users to toggle between all games (seasons 2016 thru 2020) either a regular season game or a play-off match and access all shot/goal events of the corresponding games to display the punk position on the rink. 

In [101]:
img_folder_path = 'C:\\Users\\chait\\OneDrive\\Documents\\School\\IFT_6758\\2.Project\\Milestone1\\ift6758\\ift6758-project-template-main\\figures\\'

In [216]:
# Game id is argumented in this function

df['game_id_numeric'] = pd.to_numeric(df['game_id'])
list_game_id = df['game_id_numeric'].unique().tolist()

def Game_ID(var):
    global cnt
    global list_Event_Id
    global check, display
    check = var
    cnt = -1
    list_Event_Id = []
    with open(f'{data_folder_path}{var}.json','r') as f:
        data = json.load(f)
        df = pd.DataFrame(data)
        for i in df['liveData']['plays']['allPlays']:
                if i['result']['event'] == 'Shot' or i['result']['event'] == 'Goal':
                    cnt += 1
                    list_Event_Id.append(i['about']['eventIdx']) 
x = widgets.interact(Game_ID, var=widgets.SelectionSlider(description='Game_ID',options=list_game_id))


interactive(children=(SelectionSlider(description='Game_ID', options=(2018020001, 2018020002, 2018020003, 2018…

In [217]:
#Event id (Shot/Goal) is argumented for a selected game to locate punk on rink 

def Event_ID(val):
    global temp
    temp = val
    df_group = df.loc[df['game_id'] == check]
    df_group = df_group.loc[df['event_id'] == temp]
    player=df_group['attacking_player'].tolist()
    team=df_group['attacking_team'].tolist()
    period=df_group['period'].tolist()
    type=df_group['shot_type'].tolist()
    
    img = plt.imread(f'{img_folder_path}nhl_rink.png') 
    fig = plt.figure(figsize=(35,15))
    fig.suptitle(f'Game {check}:{type[0]} by {player[0]} of {team[0]} in period {period[0]}',fontsize=25,y=.95)
    ax1 = fig.add_subplot(1,1,1)
    ax1.tick_params(labelsize=25)
    ax1.imshow(img,extent=[-100,100,-42.5,42.5])
    ax1.plot(df_group['x_coordinates'],df_group['y_coordinates'],'b.',markersize=50)
    ax1.set_title("Shot Locator",fontsize=30)
    ax1.set_xlim(-100,100)
    ax1.set_ylim(-42.5,42.5)
    
y = widgets.interact(Event_ID, val=widgets.SelectionSlider(description='Event_ID',options=list_Event_Id))

interactive(children=(SelectionSlider(description='Event_ID', options=(7, 14, 15, 16, 22, 27, 30, 36, 39, 41, …